# Исследование надёжности заёмщиков

Заказчик исследования — кредитный отдел банка. Нужно разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. Входные данные от банка — статистика о платёжеспособности клиентов.
Результаты исследования будут учтены при построении модели кредитного скоринга — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.


Описание данных  
children — количество детей в семье   
days_employed — общий трудовой стаж в днях   
dob_years — возраст клиента в годах   
education — уровень образования клиента   
education_id — идентификатор уровня образования   
family_status — семейное положение   
family_status_id — идентификатор семейного положения   
gender — пол клиента   
income_type — тип занятости   
debt — имел ли задолженность по возврату кредитов   
total_income — ежемесячный доход   
purpose — цель получения кредита

## 1. Обзор данных

In [1]:
import pandas as pd

In [2]:
try:
    data = pd.read_csv('C:\\Users\\katrusya\\Desktop\\Датасеты для Яндекс практикума\\data_reliability_of_borrowers.csv')
except:
    print("can't open the file")

In [3]:
data.head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу


In [4]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     19351 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      19351 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


## 2.Предобработка данных

In [5]:
min(data['days_employed'].unique())


-18388.949900568383

Здесь, как и стоило ожидать, отрицательные значения. Заменим их модулем

In [6]:
data['days_employed'] = abs(data['days_employed'])
min(data['days_employed'])

24.14163324048118

Похоже, с этой аномалией справились. Посмотрим, что у нас в стобце о детях.

In [7]:
data['children'].unique()

array([ 1,  0,  3,  2, -1,  4, 20,  5], dtype=int64)

Два аномальных значения. Думаю, -1 стоит заменить модулем, а 20 выглядит подозрительно, хотя такое и возможно. Думаю, для статистики такой случай не будет необходимым, избавлюсь от него

In [8]:
data['children'] = abs(data['children'])
data = data[data['children'] != 20]
data['children'].unique()

array([1, 0, 3, 2, 4, 5], dtype=int64)

Посмотрим пропущенные значения

In [9]:
data.isna().sum()

children               0
days_employed       2165
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        2165
purpose                0
dtype: int64

Пропуски у нас присутствуют в столбцах 'days_employed' и 'total_income'. Причём и там и там их по 2165. Это говорит о том, что заполнение одного столбца влияет на наличие данных в другом. Причину можно было бы связать с психологическим фактором (т.е. опрашиваемые, по какой-то причине скрывающие информацию о количестве отработанных дней так же неохотно делятся информацией о своих доходах), но, поскольку количество пропусков совпадает строго, то, скорее всего, причина либо техническая (произошла утеря части данных по двум столбцам), либо в какой-то из версий опросников эти два столбца отсутствовали.

Cтолбец total_income — хранит данные о доходах. На сумму дохода сильнее всего влияет тип занятости, поэтому заполнить пропуски в этом столбце стоит медианным значением по каждому типу из столбца income_type.   
Медианное значение в случае с количественными переменными является более близким к реальности значением, поскольку исключает влияние аномально больших и аномально малых значений, которые могут существенно исказить картину.
Для пропусков в days_employed сделаю то же самое. Заменю пропуски медианными значениями по каждому типу занятости.

In [10]:
for t in data['income_type'].unique():
    data.loc[(data['income_type'] == t) & (data['total_income'].isna()),  'total_income'] = \
    data.loc[data['income_type'] == t, 'total_income'].median()
    data.loc[(data['income_type'] == t) & (data['days_employed'].isna()), 'days_employed'] = \
    data.loc[data['income_type'] == t, 'days_employed'].median

In [11]:
data.isna().sum()

children            0
days_employed       0
dob_years           0
education           0
education_id        0
family_status       0
family_status_id    0
gender              0
income_type         0
debt                0
total_income        0
purpose             0
dtype: int64

Готово. Теперь, думаю стоит заменить вещественный тип данных в столбце "total_income" на целочисленный

In [12]:
data['total_income'] = data['total_income'].astype(int)
data['total_income'].dtypes

dtype('int32')

In [13]:
data['education'].unique()

array(['высшее', 'среднее', 'Среднее', 'СРЕДНЕЕ', 'ВЫСШЕЕ',
       'неоконченное высшее', 'начальное', 'Высшее',
       'НЕОКОНЧЕННОЕ ВЫСШЕЕ', 'Неоконченное высшее', 'НАЧАЛЬНОЕ',
       'Начальное', 'Ученая степень', 'УЧЕНАЯ СТЕПЕНЬ', 'ученая степень'],
      dtype=object)

Наблюдаю неявные дубликаты. Нужно привести названия к нижнему регистру и избавиться от дубликатов.

In [14]:
data['education'] = data['education'].str.lower()
data['education'].unique()

array(['высшее', 'среднее', 'неоконченное высшее', 'начальное',
       'ученая степень'], dtype=object)

In [15]:
data.duplicated().sum()

71

In [16]:
data = data.drop_duplicates()
data.duplicated().sum()

0

Посмотрю, нет ли отрицательных значений дохода

In [17]:
data['total_income'].min()

20667

###  Категоризация данных

Присвою категорию, на основании уровня дохода и создам новый столбец 'total_income_category'.
- 0–30000 — `'E'`;
- 30001–50000 — `'D'`;
- 50001–200000 — `'C'`;
- 200001–1000000 — `'B'`;
- 1000001 и выше — `'A'`.


In [18]:
def categorize_income(income):
    try:
        if income < 30001:
            return 'E'
        if income < 50001:
            return 'D'
        if income < 200001:
            return 'C'
        if income < 1000001:
            return 'B'
        return 'A'
    except:
        pass

In [19]:
data['total_income_category'] = data['total_income'].apply(categorize_income)
data.head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,total_income_category
0,1,8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья,B
1,1,4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля,C
2,0,5623.42261,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья,C
3,3,4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование,B
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу,C


In [20]:
data['purpose'].unique()

array(['покупка жилья', 'приобретение автомобиля',
       'дополнительное образование', 'сыграть свадьбу',
       'операции с жильем', 'образование', 'на проведение свадьбы',
       'покупка жилья для семьи', 'покупка недвижимости',
       'покупка коммерческой недвижимости', 'покупка жилой недвижимости',
       'строительство собственной недвижимости', 'недвижимость',
       'строительство недвижимости', 'на покупку подержанного автомобиля',
       'на покупку своего автомобиля',
       'операции с коммерческой недвижимостью',
       'строительство жилой недвижимости', 'жилье',
       'операции со своей недвижимостью', 'автомобили',
       'заняться образованием', 'сделка с подержанным автомобилем',
       'получение образования', 'автомобиль', 'свадьба',
       'получение дополнительного образования', 'покупка своего жилья',
       'операции с недвижимостью', 'получение высшего образования',
       'свой автомобиль', 'сделка с автомобилем',
       'профильное образование', 'высшее об

Я бы разделила все цели получения кредита на следующие категории:    
- 'операции с автомобилем',
- 'операции с недвижимостью',
- 'проведение свадьбы',
- 'получение образования'.
и создала новый столбец 'purpose_category'

In [21]:
def categorize_purpose(operation):
    try:
        if ("жил" in operation) | ('недвиж' in operation):
            return "операции с недвижимостью"
        elif ("автомобил" in operation):
            return "операции с автомобилем"
        elif ("свадьб" in operation):
            return "проведение свадьбы"
        elif ('образован' in operation):
            return "получение образования"
    except:
        return "без категории"

In [22]:
data['purpose_category'] = data['purpose'].apply(categorize_purpose)

In [23]:
data.head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,total_income_category,purpose_category
0,1,8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья,B,операции с недвижимостью
1,1,4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля,C,операции с автомобилем
2,0,5623.42261,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья,C,операции с недвижимостью
3,3,4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование,B,получение образования
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу,C,проведение свадьбы


## 3. Изучаю зависимость выплаты кредитов от различных факторов

Для того, чтобы изучать зависимость от различных факторов, напишу функцию, которая будет возвращать отсортированную сводную таблицу, отражающую процент задолженности в зависимости от заданного фактора.

In [24]:
def debt_factor(factor):
    data_pivot_factor = data.pivot_table(index=factor, values='debt', aggfunc={'count', 'sum', 'mean'})
    #поскольку в 'debt' только 0 и 1, добавила mean, это сразу покажет статистику по задолженности.
    data_pivot_factor['loan_repay'] = 100 - data_pivot_factor['mean'] * 100
    data_pivot_factor= data_pivot_factor.sort_values(by='loan_repay', ascending=False)
    return data_pivot_factor

### Eсть ли зависимость между количеством детей и возвратом кредита в срок?

In [25]:
data_pivot_children = debt_factor('children')
data_pivot_children

,count,mean,sum,loan_repay
children,,,,
5,9,0.000000,0,100.000000
0,14091,0.075438,1063,92.456178
3,330,0.081818,27,91.818182
1,4855,0.091658,445,90.834192
2,2052,0.094542,194,90.545809
4,41,0.097561,4,90.243902


Глядя на таблицу, можно сказать, что ярко выраженной зависимости между количеством детей у заёмщика и вероятностью возврата кредита не наблюдается. Лучше всего возвращают долги заёмщики с 5 детьми, но вряд ли 9 человек могут представлять статистически значимую картину. У остальных представителей процент возврата колеблется в районе 90.2% - 92.5%. Семьи без детей являются наиболее стабильными из семей с количеством детей до 4 включительно. Видимо сказывается более независимое положение и более низкая вероятность непредвиденных трат. Семьи же с 4-мя детьми наоборот имеют самый низкий процент возврата денежных средств. Вероятно, что кредит в этих случаях чаще берется по причинам срочной необходимости (например, семья просто не умещается на квадратных метрах). Кроме того, в таких семьях вероятность непредвиденных трат прямо связана с количеством детей (например на лечение, образование и т.д). Но, опять-таки, эта разница незначительна. 

### Есть ли зависимость между семейным положением и возвратом кредита в срок?

Проведу аналогичное исследование относительно семейного положения

In [26]:
data_pivot_family_status = debt_factor('family_status')
data_pivot_family_status

,count,mean,sum,loan_repay
family_status,,,,
вдовец / вдова,955,0.065969,63,93.403141
в разводе,1193,0.070411,84,92.958927
женат / замужем,12290,0.075509,928,92.449146
гражданский брак,4139,0.093018,385,90.698236
Не женат / не замужем,2801,0.097465,273,90.253481


Из сводной таблицы можно сделать вывод о том, что самый высокий процент возврата долгов у вдовцов/вдов. Немногим меньше процент возврата у тех кто был или находится в браке. Скорее всего это более возрастная категория по сравнению с теми, кто находится в гражданском браке либо никогда в нём не состоял. Т.е, вероятно, доход у них в среднем выше, финансовое положение - стабильнее, опыт и степень отвественности тоже выше.

### Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

In [27]:
data_pivot_income = debt_factor('total_income_category')
data_pivot_income

,count,mean,sum,loan_repay
total_income_category,,,,
D,350,0.060000,21,94.000000
B,5022,0.070490,354,92.951016
A,25,0.080000,2,92.000000
C,15959,0.084842,1354,91.515759
E,22,0.090909,2,90.909091


Мы видим, что хуже всех возвращают долг клиенты с самым низким доходом. Это логично и ожидаемо. Лучше всех возвращают деньги люди с невысоким стабильным доходом. Скорее всего это клиенты, привыкшие строго контролировать свои расходы и менее склонные к стихийным тратам. Клиенты с самым высоким доходом менее охотно возвращают долги, чем клиенты с доходом категории "В". Возможно это связано с тем, что доход у них не всегда стабилен. Если даже у клиента, относящегося к категории "А" стабильная работа, то, в случае увольнения, подобрать работу с таким же уровнем дохода непросто. Но, надо сказать, что группы 'D', 'A', 'E' статистической значимости не представляют, для их анализа нужно иметь более объемные группы.

### Как разные цели кредита влияют на его возврат в срок

In [28]:
data_pivot_purpose = debt_factor('purpose_category')
data_pivot_purpose

,count,mean,sum,loan_repay
purpose_category,,,,
операции с недвижимостью,10775,0.072390,780,92.761021
проведение свадьбы,2315,0.079050,183,92.095032
получение образования,3998,0.092296,369,90.770385
операции с автомобилем,4290,0.093473,401,90.652681


Из таблицы видно, что лучше всего возвращают долги по кредиту в случае операций с недвижимостью. Видимо это самый ответственный сектор клиентов, который определился с местом жительства и не будет тратиться на съем жилья. Либо, если это жилье не первое, то сдача его в аренду может частично или полностью погашать кредит, снижая тем самым риск задолженности клиента. Кроме того, кредиты по недвижимости выплачиваются часто не одним человеком, а домохозяйством.   
Проведение свадьбы хотя и является достаточно легкомысленным поводом для взятия кредита, но, видимо сумма кредита в этом случае гораздо ниже, а, следовательно быстрее погашается. Кроме того, специфика нашей страны предполагает, что многие гости на свадьбу дарят деньги, которые могут пойти на погашение кредита.   
Автомобиль часто не является необходимой покупкой, кроме того покупка автомобиля тянет за собой затраты на его обслуживание, в том числе внезапные и крупные. Автомобиль редко бывает связан с получением дохода в отличие от недвижимости, кроме того, его цена обычно падает с возрастом. Видимо это является причиной того, что клиенты, которые берут кредит на покупку авто, в среднем хуже возвращают деньги.

## 4.Вывод


Самый большой объём данных представлен в семьях, включающих от 0 до 2х детей.
По результатам самыми добросовестными заёмщиками являются клиенты без детей, они возвращают около 92,5% кредитов без задолжености, семьи с одним ребёнком возвращают около 90,8%, с двумя - около 90,6%.
___
Что касается семейного положения, то тут самыми надёжными являются те клиенты, которые состоят или когда-либо состояли в браке:
вдовые возвращают в срок в 93,4% случаев, разведенные в 92,9%, состоящие в браке в 92,4%. В это же время статистика по клиентам не состоявшим когда-либо в браке следующая: находящиеся в гражданском браке возвращают в 90,7% случаев, не состоящие в браке  - в 90,2%.
___
Относительно уровней дохода самый большой объем данных представлен в категориях с уровнем дохода от 30000 до 1000000. Среди этих клиентов банка самыми добросовестными заёмщиками оказались люди с уровнем дохода в диапазоне от 30000 до 50000. Они возвращают вовремя почти в 94,0% случаев. Заемщики, уровень дохода которых составляет от 200000 до 1000000, возвращают почти в 93,0% случаев. И, наконец, клиенты, с уровнем дохода от 50000 до 200000, возвращают в 91,5%. 
___
Касаемо цели кредита ситуация следующая. Лучше всего возвращают кредиты, взятые на операции с недвижимостью - 92,7%, проведение свадьбы - 92,1%. Хуже статистика по получению образования и операциям с автомобилями. Здесь доля возврата составляет около 90,7%.
___
___

По совокупности для банка наиболее надёжными клиентами являются клиенты:
без детей,
вдовые либо в браке/разведённые,
с уровнем дохода от 30000 до 50000, 
берущие кредит на операции с недвижимостью.

Самый рискованный вариант клиента в плане возврата кредита:
человек двумя детьми,
не состоящий в браке либо состоящий в гражданском браке,
берущий кредит на операцию с автомобилем.